In [16]:
# Librerías Jaime Estrada
import requests
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


enlace = "https://raw.githubusercontent.com/selva86/datasets/master/Advertising.csv"
archivo_salida = Path("datos.csv")

cabecera = {"User-Agent": "descargador-simple/1.0"}

try:

    with requests.get(enlace, headers=cabecera, stream=True, timeout=15) as resp:
        resp.raise_for_status()
        with archivo_salida.open("wb") as archivo:
            for parte in resp.iter_content(chunk_size=8192):
                if parte:
                    archivo.write(parte)

    print(f"Archivo descargado en: {archivo_salida.resolve()}")


    datos = pd.read_csv(archivo_salida)

    print("\nPrimeras filas del dataset:")
    print(datos.head(), "\n")

    print("Información general del dataset:\n")
    print(datos.info())

except Exception as error:
    print("Error al descargar o leer el archivo:", error)
    raise SystemExit


# === PREPARACIÓN DE DATOS ===

X = datos[["TV", "radio", "newspaper"]]
y = datos["sales"]


X_ent, X_pru, y_ent, y_pru = train_test_split(X, y, test_size=0.2, random_state=42)

# === MODELOS A EVALUAR ===
modelos = {
    "Lineal": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1)
}

resultados = {}

# === ENTRENAMIENTO Y EVALUACIÓN ===
for nombre, modelo in modelos.items():
    modelo.fit(X_ent, y_ent)
    pred = modelo.predict(X_pru)
    # Cálculo de métricas
    resultados[nombre] = {
        "MAE": mean_absolute_error(y_pru, pred),
        "RMSE": mean_squared_error(y_pru, pred) ** 0.5,
        "R²": r2_score(y_pru, pred)
    }


print("\nResultados de los modelos:\n")
for nombre, metricas in resultados.items():
    print(f"Modelo: {nombre}")
    for metrica, valor in metricas.items():
        print(f"  {metrica}: {valor:.4f}")
    print()


Archivo descargado en: /content/datos.csv

Primeras filas del dataset:
   Unnamed: 0     TV  radio  newspaper  sales
0           1  230.1   37.8       69.2   22.1
1           2   44.5   39.3       45.1   10.4
2           3   17.2   45.9       69.3    9.3
3           4  151.5   41.3       58.5   18.5
4           5  180.8   10.8       58.4   12.9 

Información general del dataset:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB
None

Resultados de los modelos:

Modelo: Lineal
  MAE: 1.4608
  RMSE: 1.7816
  R²: 0.8994

Modelo: Ridge
  MAE: 1.4608
  RMSE: 1.7816
  R²: 0.8994

Modelo: Lasso
  MAE: 1.4598